# Tenencia del producto préstamo en el DataSet de Banca Checo  

In [ ]:
import pandas as pd
%pylab inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy.core.multiarray

In [ ]:
#Para garantizar la replicabilidad del análisis
np.random.seed()

In [ ]:
url="https://raw.githubusercontent.com/osmaac/master-data-science/master/DFTenenciaProductos.csv"
df_original=pd.read_csv(url)

In [ ]:
#Comenzamos a revisar que el DF se haya importado correctamente
df_original.shape

In [ ]:
df_original.head()

In [ ]:
# Vamos a analizar si hay missings al cargar los datos a Python
df_original.columns[df_original.isnull().sum()!=0]
#Vemos que las variables con missings provienen de variables que ya tenían esos missings en el DataFrame generado con R,
#ya que el disponent (autorizado), los préstamos y las tarjetas no son productos que tengan asociados 
#todas las cuentas (sólo las cuentas que los hayan contratado) 

In [ ]:
#Vemos como se han importado las variables del DataFrame de R al DataFrame que vamos a utilizar en Python 
print(df_original.iloc[:,0:32].dtypes)
print(df_original.iloc[:,31:61].dtypes)
print(df_original.iloc[:,60:70].dtypes)
#Observamos que las variables de factor y de fecha han modificado su tipo de variable, 
#por lo que tendremos que trabajar con ellas

In [ ]:
#Transformamos a formato fecha las variables que originalmente eran fecha en R
df_original[["Date_Account","birth_owner", "birth_disponent", "Date_Loan", "owner_card_date"]]=df_original[["Date_Account","birth_owner", "birth_disponent", "Date_Loan", "owner_card_date"]].apply(pd.to_datetime)
from datetime import datetime
df_original['Date_Account']=df_original['Date_Account'].apply(datetime.toordinal)
df_original['birth_owner']=df_original['birth_owner'].apply(datetime.toordinal)
df_original['birth_disponent']=df_original['birth_disponent'].apply(datetime.toordinal)
df_original['Date_Loan']=df_original['Date_Loan'].apply(datetime.toordinal)
df_original['owner_card_date']=df_original['owner_card_date'].apply(datetime.toordinal)

In [ ]:
plt.hist(df_original['crimes_95'])

In [ ]:
plt.hist(df_original['crimes_95_ratio'])

In [ ]:
plt.hist(df_original['crimes_96'])

In [ ]:
plt.hist(df_original['crimes_96_ratio'])

In [ ]:
plt.hist(df_original['entrepreneurs'])

In [ ]:
plt.hist(df_original['entrepreneurs_ratio'])

In [ ]:
#Creamos un DataFrame con las variables que vamos a considerar numéricas
df_num=df_original[['Date_Account','birth_owner', 'birth_disponent','owner_card_date','Ord_Insurance', 'Ord_Insurance_amount',
                    'Ord_Household_Payment','Ord_Household_Payment_amount', 'Ord_Loan_Payment', 'Ord_Leasing',
                    'Ord_Empty', 'Ord_Empty_amount', 'num_inhabitants', 'municip < 499', 'municip 500-1999',
                    'municip 2000-9999', 'municip > 10000', 'num_cities', 'avg_salary',  
                    'Num_Type_Credit', 'Num_Type_VYBER', 'Num_Type_Withdrawal', 'Num_Op_Null', 'Num_Op_Remittances',
                    'Num_Op_Collection','Num_Op_CashCredit', 'Num_Op_WithdrawalCash','Num_Op_WithdrawalCreditCard',
                    'Num_Sym_Null', 'Num_Sym_Null2','Num_Sym_Pension', 'Num_Sym_Insurance', 'Num_Sym_NegBal',
                    'Num_Sym_Household', 'Num_Sym_Statement', 'Num_Sym_IntDep', 'Num_Sym_LoanPayment', 
                    'Balance_in_negative','Ord_Loan_Payment_amount', 'Ord_Leasing_amount','ratio_urban_inhabitants',
                    'unemployment_rate_95','unemployment_rate_96', 'crimes_95_ratio', 'crimes_96_ratio', 'entrepreneurs_ratio' ]] 

In [ ]:
#Creamos un DataFrame con las variables que vamos a considerar categóricas
df_cat=df_original[['account_disponent_bin','frequency', 'sex_owner', 'owner_card_type',
       'sex_disponent']]
#Vemos que tipos tienen las variables que queremos que sean categóricas
df_cat.dtypes

In [ ]:
#Ponemos las variables "owner_card_type" y "account_disponent_bin" como string para poder obtener dummies
df_cat["owner_card_type"]=df_cat["owner_card_type"].astype(str)
df_cat["account_disponent_bin"]=df_cat["account_disponent_bin"].astype(str)

In [ ]:
df_cat_dumm=pd.get_dummies(df_cat)

In [ ]:
df_cat_dumm.dtypes
#Al pasar a dummies las variables, hemos incrementado en 8 el número total de variables

In [ ]:
df = pd.concat([df_num, df_cat_dumm], axis = 1)
df.head()

# Vamos a generar un primer modelo benchmark y vamos a ver si podemos obtener una simplificación de los datos con PCA

In [ ]:
X = df
y = df_original["account_loan_bin"] 

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2).fit_transform(X)
plt.figure(dpi=120)
plt.scatter(pca[y.values==0,0], pca[y.values==0,1], alpha=0.5, label='NO', s=2, color='navy')
plt.scatter(pca[y.values==1,0], pca[y.values==1,1], alpha=0.5, label='YES', s=2, color='darkorange')
plt.legend()
plt.title('Producto préstamo')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
#Del gráfico anterior no consigo sacar nada en claro
pca2 = PCA(n_components=2)
pca2.fit(X)
print(pca2.components_)
print(pca2.explained_variance_ratio_)
#De los componentes de momento tampoco sacamos ninguna conclusión. Las variables están en escalas muy distintas y por eso 
#el PCA genera resultados tan "positivos" en cuanto a explicabilidad de las 2 primeras componentes principales (74%)

In [ ]:
#Empezamos con modelos sencillos. Estos modelos han de predecir si una cuenta (account) contrata préstamo (loan=1) o no (loan=0)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
#Generamos conjuntos de train y de test. Para el test usamos el 20% de las observaciones
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
#Probamos una Regresión logística
clf_LR=LogisticRegression(random_state=0) #vamos a incluir Random State en todos los modelos para poder reproducirlos
clf_LR.fit(X_train,y_train)

In [ ]:
#Creamos una función para poder evaluar el modelo, ver si genera overfitting, underfitting y tener detallada la matriz de
#confusión de los modelos que generemos
def eval_modelo (clf,X_train,y_train, X_test,y_test):
    print("Datos de train:")
    print("El accuracy es",accuracy_score(y_train,clf.predict(X_train))*100,"%")
    print("La precision es",precision_score(y_train,clf.predict(X_train))*100, "%")
    print("El recall es",recall_score(y_train,clf.predict(X_train))*100,"%")
    tn, fp, fn, tp=confusion_matrix(y_train,clf.predict(X_train)).ravel()
    print("tn:",tn," fp:",fp," fn:",fn," tp:",tp)
    print("Datos de test:")
    print("El accuracy es",accuracy_score(y_test,clf.predict(X_test))*100,"%")
    print("La precision es",precision_score(y_test,clf.predict(X_test))*100, "%")
    print("El recall es",recall_score(y_test,clf.predict(X_test))*100,"%")
    tn_t, fp_t, fn_t, tp_t=confusion_matrix(y_test,clf.predict(X_test)).ravel()
    print("tn:",tn_t," fp:",fp_t," fn:",fn_t," tp:",tp_t)      

In [ ]:
eval_modelo(clf_LR,X_train,y_train, X_test,y_test)

In [ ]:
#Probamos con un árbol de decisión
clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)
clf_tree.fit(X_train,y_train)

In [ ]:
eval_modelo(clf_tree,X_train,y_train, X_test,y_test)

In [ ]:
#Las clases a predecir (si la cuenta tiene prestamo=1 ó no tiene =0) están desbalanceadas
y2=pd.DataFrame(y)
sns.countplot(x="account_loan_bin",data=y2, palette='hls')
plt.show

In [ ]:
y2['account_loan_bin'].value_counts()

In [ ]:
#Veamos el porcentaje que representa cada clase:
print("Las cuentas CON préstamo son el", "%.2f" % (y2['account_loan_bin'].value_counts()[0]/len(y2['account_loan_bin'])*100) ,"%")
print("Las cuentas SIN préstamo son el", "%.2f" % (y2['account_loan_bin'].value_counts()[1]/len(y2['account_loan_bin'])*100) ,"%")

In [ ]:
#Vamos a querer dibujar Decision Trees. Importamos los paquetes necesarios para dibujarlos
from io import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus

In [ ]:
#Función para dibujar un árbol
def dibu_arb(tree):
    dot_data = StringIO()
    export_graphviz(tree, out_file=dot_data,filled=True, rounded=True,
                special_characters=True)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
    return(Image(graph.create_png()))
    

In [ ]:
dibu_arb(clf_tree)

In [ ]:
X_train.columns[36]
#Parece que la variable "Num_Sym_LoanPayment" contiene toda la información de "account_loan_bin", aunque se han extraido 
#de ficheros distintos. Pero por el nombre se puede intuir que es así. 

In [ ]:
X.columns[clf_tree.feature_importances_>0.10] #Vamos a ver las variables más importantes para el modelo

In [ ]:
clf_tree.feature_importances_

In [ ]:
#Link interesante para ver como se realiza el cálculo de "feature_importances"_
#https://medium.com/@srnghn/the-mathematics-of-decision-trees-random-forest-and-feature-importance-in-scikit-learn-and-spark-f2861df67e3

In [ ]:
# conda install -c glemaitre imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTENC
#Hemos tomado SMOTENC y no SMOTE para poder aplicar el algoritmo a variables categóricas

In [ ]:
#Para ver las columnas que vamos a denominar como categóricas cuando apliquemos SMOTE
print(X_train.columns)

In [ ]:
X_train.shape

In [ ]:
smo=SMOTENC(categorical_features=range(46,59),random_state=0)#Las variables categóricas van a ser de la 46 a la 59
os_X,os_y=smo.fit_sample(X_train, y_train)
columns = X_train.columns
os_X = pd.DataFrame(data=os_X,columns=columns)
os_y= pd.DataFrame(data=os_y,columns=['account_loan_bin'])

In [ ]:
#Chequeamos que SMOTENC funciona como esperábamos

print("length of oversampled data is ",len(os_X))
print("Number of loans=0 in oversampled data",len(os_y[os_y['account_loan_bin']==0]))
print("Number of loans=1",len(os_y[os_y['account_loan_bin']==1]))
print("Proportion of loans=0 is ",len(os_y[os_y['account_loan_bin']==0])/len(os_X))
print("Proportion of loans=1 is ",len(os_y[os_y['account_loan_bin']==1])/len(os_X))

os_bin=os_X[['account_disponent_bin_0','account_disponent_bin_1',
       'frequency_After_trans', 'frequency_Monthly', 'frequency_Weekly',
       'sex_owner_F', 'sex_owner_M', 'owner_card_type_0', 'owner_card_type_1',
       'owner_card_type_2', 'owner_card_type_3', 'sex_disponent_F',
       'sex_disponent_M']]

print("unique de variables categóricas",unique(os_bin))
print("unique de variable y",unique(os_y))

In [ ]:
os_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)
clf_tree_os=os_clf_tree.fit(os_X,os_y)

In [ ]:
eval_modelo(clf_tree_os,os_X,os_y, X_test, y_test)
#Obtenemos los mismos resultados con y sin SMOTE. Lógicamente porque estamos suponiendo que la variable 
#'Num_Sym_LoanPayment' contiene la información de si la cuenta tiene un préstamo o no

In [ ]:
#Vemos que tener un préstamo correlaciona de forma muy significativa con las variables:Ord_Loan_Payment,Num_Sym_LoanPayment y 
#Ord_Loan_Payment_amount
df_original.corr()["account_loan_bin"]

In [ ]:
#Vemos las correlaciones de las variables anteriores
print(np.corrcoef(df_original["account_loan_bin"],df_original["Ord_Loan_Payment"]))
print(np.corrcoef(df_original["account_loan_bin"],df_original["Ord_Loan_Payment_amount"]))
print(np.corrcoef(df_original["account_loan_bin"],df_original["Num_Sym_LoanPayment"]))

In [ ]:
#Vamos a ver qué resultados obtenemos con los modelos eliminando la variable 'Num_Sym_LoanPayment' (la que anteriormente había
#salido como más relevante). Generamos un nuevo DataFrame
X1=X.drop(['Num_Sym_LoanPayment'], axis=1)

In [ ]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size=0.2, random_state=0)

In [ ]:
X1_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)
clf_treeX1=X1_clf_tree.fit(X1_train,y1_train)

In [ ]:
#Aplicamos oversampling
smo=SMOTENC(categorical_features=range(45,58),random_state=0)# Modificamos rango porque hemos eliminado una variable
os_X1,os_y1=smo.fit_sample(X1_train, y1_train)
columns = X1_train.columns
os_X1 = pd.DataFrame(data=os_X1,columns=columns)
os_y1= pd.DataFrame(data=os_y1,columns=['account_loan_bin'])

In [ ]:
#Chequeamos que SMOTENC funciona como esperábamos

print("length of oversampled data is ",len(os_X1))
print("Number of loans=0 in oversampled data",len(os_y1[os_y1['account_loan_bin']==0]))
print("Number of loans=1",len(os_y1[os_y1['account_loan_bin']==1]))
print("Proportion of loans=0 is ",len(os_y1[os_y1['account_loan_bin']==0])/len(os_X1))
print("Proportion of loans=1 is ",len(os_y1[os_y1['account_loan_bin']==1])/len(os_X1))

os_bin=os_X1[['account_disponent_bin_0','account_disponent_bin_1',
       'frequency_After_trans', 'frequency_Monthly', 'frequency_Weekly',
       'sex_owner_F', 'sex_owner_M', 'owner_card_type_0', 'owner_card_type_1',
       'owner_card_type_2', 'owner_card_type_3', 'sex_disponent_F',
       'sex_disponent_M']]
print("unique de variables categóricas",unique(os_bin))
print("unique de variable y",unique(os_y1))

In [ ]:
osX1_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)
clf_tree_osX1=osX1_clf_tree.fit(os_X1,os_y1)

In [ ]:
#Evaluamos los modelos
print("Sin Oversampling")
eval_modelo(clf_treeX1,X1_train,y1_train, X1_test, y1_test)
print("Con Oversampling")
eval_modelo(clf_tree_osX1,os_X1,os_y1, X1_test, y1_test)

In [ ]:
dibu_arb(clf_treeX1)

In [ ]:
dibu_arb(clf_tree_osX1)

In [ ]:
X1_train.columns[37]

In [ ]:
X1.columns[clf_tree_osX1.feature_importances_>0.10]

In [ ]:
X1.columns[clf_treeX1.feature_importances_>0.10]

In [ ]:
X2=X1.drop(['Ord_Loan_Payment_amount'], axis=1)

In [ ]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y, test_size=0.2, random_state=0)

In [ ]:
#Generamos el modelo SIN oversampling
X2_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)
clf_treeX2=X2_clf_tree.fit(X2_train,y2_train)

In [ ]:
#Aplicamos oversampling
smo=SMOTENC(categorical_features=range(44,57),random_state=0)# Modificamos rango porque hemos eliminado una nueva variable
os_X2,os_y2=smo.fit_sample(X2_train, y2_train)
columns = X2_train.columns
os_X2 = pd.DataFrame(data=os_X2,columns=columns)
os_y2= pd.DataFrame(data=os_y2,columns=['account_loan_bin'])

In [ ]:
#Generamos el árbol con el DataFrame al hemos aplicado oversampling
osX2_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)
clf_tree_osX2=osX2_clf_tree.fit(os_X2,os_y2)

In [ ]:
#Evaluamos los modelos
print("Sin Oversampling")
eval_modelo(clf_treeX2,X2_train,y2_train, X2_test, y2_test)
print("Con Oversampling")
eval_modelo(clf_tree_osX2,os_X2,os_y2, X2_test, y2_test)

In [ ]:
dibu_arb(clf_treeX2)

In [ ]:
dibu_arb(clf_tree_osX2)

In [ ]:
X2_train.columns[8]

In [ ]:
X2.columns[clf_treeX2.feature_importances_>0.10]

In [ ]:
X2.columns[clf_tree_osX2.feature_importances_>0.10]

In [ ]:
#Generamos el conjunto X3, a partir de X2, eliminando la variable 'Ord_Loan_Payment'
X3=X2.drop(['Ord_Loan_Payment'], axis=1)

In [ ]:
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y, test_size=0.2, random_state=0)

In [ ]:
#Generamos el modelo SIN oversampling
X3_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)
clf_treeX3=X3_clf_tree.fit(X3_train,y3_train)

In [ ]:
#Aplicamos oversampling
smo=SMOTENC(categorical_features=range(43,56),random_state=0)# Modificamos rango porque hemos eliminado una nueva variable
os_X3,os_y3=smo.fit_sample(X3_train, y3_train)
columns = X3_train.columns
os_X3 = pd.DataFrame(data=os_X3,columns=columns)
os_y3= pd.DataFrame(data=os_y3,columns=['account_loan_bin'])

In [ ]:
#Generamos el modelo CON oversampling
osX3_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)
clf_tree_osX3=osX3_clf_tree.fit(os_X3,os_y3)

In [ ]:
#Evaluamos los modelos
print("Sin Oversampling")
eval_modelo(clf_treeX3,X3_train,y3_train, X3_test, y3_test)
print("Con Oversampling")
eval_modelo(clf_tree_osX3,os_X3,os_y3, X3_test, y3_test)

In [ ]:
dibu_arb(clf_treeX3)

In [ ]:
dibu_arb(clf_tree_osX3)

In [ ]:
print(X3_train.columns[19])#Número de un tipo especial de reintegros
print(X3_train.columns[22])#Número de transferencias enviadas

In [ ]:
X3.columns[clf_treeX3.feature_importances_>0.10]

In [ ]:
X3.columns[clf_tree_osX3.feature_importances_>0.10]

In [ ]:
clf_treeX3.feature_importances_

In [ ]:
sum(clf_treeX3.feature_importances_)

In [ ]:
print(df_original.corr()["account_loan_bin"])

In [ ]:
X4=X3.drop(['Num_Type_VYBER'], axis=1)

In [ ]:
X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y, test_size=0.2, random_state=0)

In [ ]:
#Generamos el modelo SIN oversampling
X4_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)
clf_treeX4=X4_clf_tree.fit(X4_train,y4_train)

In [ ]:
#Aplicamos oversampling
smo=SMOTENC(categorical_features=range(42,55),random_state=0)# Modificamos rango porque hemos eliminado una variable
os_X4,os_y4=smo.fit_sample(X4_train, y4_train)
columns = X4_train.columns
os_X4 = pd.DataFrame(data=os_X4,columns=columns)
os_y4= pd.DataFrame(data=os_y4,columns=['account_loan_bin'])

In [ ]:
#Generamos el modelo CON oversampling
osX4_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)
clf_tree_osX4=clf_tree.fit(os_X4,os_y4)

In [ ]:
#Evaluamos los modelos
print("Sin Oversampling")
eval_modelo(clf_treeX4,X4_train,y4_train, X4_test, y4_test)
print("Con Oversampling")
eval_modelo(clf_tree_osX4,os_X4,os_y4, X4_test, y4_test)

# Considero que el oversampling no me está aportando nada y dejo de utilizarlo en el resto del Notebook

In [ ]:
dibu_arb(clf_treeX4)

In [ ]:
dibu_arb(clf_tree_osX4)

# Trabajamos a continuación sobre las variables con las que nos hemos quedado

In [ ]:
#Vamos a probar Robust Scaling, partiendo del DataFrame X3 (después de eliminar las variables que parece que contenían 
#la información de si una cuenta ha contratado préstamo o no lo ha contratado)
from sklearn.preprocessing import RobustScaler
rbs = RobustScaler()
columns = X3.columns
rbs_scale = rbs.fit_transform(X3)
X=pd.DataFrame(rbs_scale,columns=columns)
X.shape

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)#Con scaling
clf_tree=X_clf_tree.fit(X_train,y_train)

In [ ]:
#Comparamos los modelos con y sin scaling (el modelo con el conjunto X3 es el mismo que generamos en el análisis
#anterior de variables)
print("Sin Scaling")
eval_modelo(clf_treeX3,X3_train,y3_train, X3_test, y3_test)
print("Con Scaling")
eval_modelo(X_clf_tree,X_train,y_train, X_test, y_test)

In [ ]:
from sklearn.preprocessing import StandardScaler
scl = StandardScaler()
columns = X3.columns
scl_scale = scl.fit_transform(X3)
X=pd.DataFrame(scl_scale,columns=columns)
X.shape

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)#Con scaling
clf_tree_=X_clf_tree.fit(X_train,y_train)

In [ ]:
#Comparamos los modelos con y sin scaling (el modelo con el conjunto X3 es el mismo que generamos en el análisis
#anterior de variables)
print("Sin Scaling")
eval_modelo(clf_treeX3,X3_train,y3_train, X3_test, y3_test)
print("Con Scaling")
eval_modelo(clf_tree,X_train,y_train, X_test, y_test)

In [ ]:
#Vamos a probar si con el scaling, cuando hacemos PCA podemos obtener una representación simple de los datos. 
#Sé que va a ser difícil obtener una representación tan sencilla del Data Frame pero lo probamos.
pca = PCA(n_components=2).fit_transform(X)
plt.figure(dpi=120)
plt.scatter(pca[y.values==0,0], pca[y.values==0,1], alpha=0.5, label='NO', s=2, color='navy')
plt.scatter(pca[y.values==1,0], pca[y.values==1,1], alpha=0.5, label='YES', s=2, color='darkorange')
plt.legend()
plt.title('Producto préstamo')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
#Del gráfico anterior no consigo sacar nada en claro
pca2 = PCA(n_components=2)
pca2.fit(X)
print(pca2.components_)
print(pca2.explained_variance_ratio_)

In [ ]:
#Standard Scaling tampoco ha funcionado mejor que sin scaling. Aplicamos scaling sólo a las variables numéricas
print(X3.columns)
print(X3.shape)

In [ ]:
#Separamos las variables numéricas de las que consideramos categóricas
X3_cat=X3[['account_disponent_bin_0','account_disponent_bin_1',
       'frequency_After_trans', 'frequency_Monthly', 'frequency_Weekly',
       'sex_owner_F', 'sex_owner_M', 'owner_card_type_0', 'owner_card_type_1',
       'owner_card_type_2', 'owner_card_type_3', 'sex_disponent_F',
       'sex_disponent_M']]
X3_num=X3[['Date_Account', 'birth_owner', 'birth_disponent', 'owner_card_date',
       'Ord_Insurance', 'Ord_Insurance_amount', 'Ord_Household_Payment',
       'Ord_Household_Payment_amount', 'Ord_Leasing', 'Ord_Empty',
       'Ord_Empty_amount', 'num_inhabitants', 'municip < 499',
       'municip 500-1999', 'municip 2000-9999', 'municip > 10000',
       'num_cities', 'avg_salary', 'Num_Type_Credit', 'Num_Type_VYBER',
       'Num_Type_Withdrawal', 'Num_Op_Null', 'Num_Op_Remittances',
       'Num_Op_Collection', 'Num_Op_CashCredit', 'Num_Op_WithdrawalCash',
       'Num_Op_WithdrawalCreditCard', 'Num_Sym_Null', 'Num_Sym_Null2',
       'Num_Sym_Pension', 'Num_Sym_Insurance', 'Num_Sym_NegBal',
       'Num_Sym_Household', 'Num_Sym_Statement', 'Num_Sym_IntDep',
       'Balance_in_negative', 'Ord_Leasing_amount', 'ratio_urban_inhabitants',
       'unemployment_rate_95', 'unemployment_rate_96', 'crimes_95_ratio',
       'crimes_96_ratio', 'entrepreneurs_ratio']]

In [ ]:
#Probamos con el Robust Scaling
columns = X3_num.columns
X3_scale = rbs.fit_transform(X3_num)
X3_scale=pd.DataFrame(X3_scale,columns=columns)
X = pd.concat([X3_scale,X3_cat], axis = 1)
X.shape

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)#Con scaling
clf_tree_=X_clf_tree.fit(X_train,y_train)

In [ ]:
#Comparamos los modelos con y sin scaling
print("Sin Scaling")
eval_modelo(clf_treeX3,X3_train,y3_train, X3_test, y3_test)
print("Con Robust Scaling sólo en variables numéricas")
eval_modelo(clf_tree,X_train,y_train, X_test, y_test)

In [ ]:
#Probamos con min-max scaling
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()

In [ ]:
columns = X3_num.columns
X3_scale = minmax.fit_transform(X3_num)
X3_scale=pd.DataFrame(X3_scale,columns=columns)
X = pd.concat([X3_scale,X3_cat], axis = 1)
X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)#Con scaling
clf_tree_=X_clf_tree.fit(X_train,y_train)

In [ ]:
#Comparamos los modelos con y sin scaling
print("Sin Scaling")
eval_modelo(clf_treeX3,X3_train,y3_train, X3_test, y3_test)
print("Con Scaling")
eval_modelo(clf_tree,X_train,y_train, X_test, y_test)

In [ ]:
print(X3_num.columns)

In [ ]:
#A partir del DataFrame X3 creamos un DataFrame que contenga el log natural de las variables numéricas
cols = X3_num.columns
X3_log=pd.DataFrame(X3_num,columns=columns)#También podríamos hacer un copy del DF

In [ ]:
#Calculamos el logaritmo de las variables que consideramos numéricas
cols=X3_num.columns
for col in cols:
    X3_log[col]=X3_log[col]+1.1 #Para evitar negativos al aplicar el logaritmo añadimos 1.1 (las variables son >=0)
    X3_log[col]=np.log(X3_log[col])

In [ ]:
X3_log.head()

In [ ]:
#Hemos diminuido la variabilidad de las features
X3_log.describe()

In [ ]:
#Creamos un DataFrame a partir del DataFrame X3 (una vez eliminadas las 3 variables que contenían la información de la variable
#a predecir), aplicando logaritmo a las variables que consideramos numéricas y dejando las categóricas igual.
X_log= pd.concat([X3_log,X3_cat], axis = 1)

In [ ]:
X_log.head()

In [ ]:
col_to_bin=['Ord_Insurance', 'Ord_Household_Payment','Ord_Leasing', 'Ord_Empty','Num_Type_Credit', 'Num_Type_VYBER',
       'Num_Type_Withdrawal', 'Num_Op_Null', 'Num_Op_Remittances','Num_Op_Collection', 'Num_Op_CashCredit', 
       'Num_Op_WithdrawalCash','Num_Op_WithdrawalCreditCard', 'Num_Sym_Null', 'Num_Sym_Null2',
       'Num_Sym_Pension', 'Num_Sym_Insurance', 'Num_Sym_NegBal','Num_Sym_Household', 'Num_Sym_Statement', 'Num_Sym_IntDep',
       'Balance_in_negative']

col_out=['Ord_Insurance_amount','Ord_Household_Payment_amount','Ord_Empty_amount', 'Ord_Leasing_amount']

In [ ]:
cols = X3.columns
X3_bin=pd.DataFrame(X3,columns=cols)
X3_bin=X3_bin.drop(col_out, axis=1)#Eliminamos las columnas de amount
X3_bin=X3_bin.drop(col_to_bin, axis=1)#Eliminamos las columnas a binarizar

In [ ]:
X3_bin.shape

In [ ]:
#Definimos el dataset que queremos binarizar
X3_to_bin=X3[['Ord_Insurance', 'Ord_Household_Payment','Ord_Leasing', 'Ord_Empty','Num_Type_Credit', 'Num_Type_VYBER',
       'Num_Type_Withdrawal', 'Num_Op_Null', 'Num_Op_Remittances','Num_Op_Collection', 'Num_Op_CashCredit', 
       'Num_Op_WithdrawalCash','Num_Op_WithdrawalCreditCard', 'Num_Sym_Null', 'Num_Sym_Null2',
       'Num_Sym_Pension', 'Num_Sym_Insurance', 'Num_Sym_NegBal','Num_Sym_Household', 'Num_Sym_Statement', 'Num_Sym_IntDep',
       'Balance_in_negative']]

In [ ]:
#Función que para cada observación de cada variable asigna 1 si el valor>0 y 0 en otro caso (no tenemos valores negativos)
def binario(x):
    if x>0:
        x=1
    else:
        x=0
    return x

In [ ]:
#Aplicamos la función anterior a las variables que deseamos binarizar 
for col in col_to_bin:
    X3_to_bin[col]=X3_to_bin[col].apply(binario)

In [ ]:
X3_to_bin=X3_to_bin.astype(str)
X3_to_bin=pd.get_dummies(X3_to_bin)

In [ ]:
#Creamos un DataFrame a partir del DataFrame X3 (una vez eliminadas las 3 variables que contenían la información de la variable
#dependiente), binarizando las variables que contaban el número de operaciones.
X_bin= pd.concat([X3_bin,X3_to_bin], axis = 1)

# Con los nuevos DataFrames de features vamos a intentar encontrar el mejor Decision Tree y ver cuales son las variables más significativas para dicho modelo, que podrían servir par establecer un custome journey del cliente

In [ ]:
#Los DataFrames con los que vamos a trabajar son:
#X_orig: El Dataframe con el que habíamos trabajado antes eliminando 3 variables, también lo llamábamos X3.
X=X3
#X_log: Es el Dataframe X_orig pero haciendo el logaritmo natural a las variables numéricas
#X_bin: Es el Dataframe X_orig pero binarizando las variables que contaban cuantas operaciones de cada tipo se habían realizado
#en cada cuenta, ahora decimos si la cuenta tiene ese tipo de operativa o no, y eliminando las variables que cuantificaban los
#importes de dicha operativa

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#Incluimos la métrica del F1 Score en la evaluación de los modelos. Modificamos la función anterior de evaluación
def eval_modelo_2 (clf,X_train,y_train, X_test,y_test):
    print("Datos de train:")
    print("El accuracy es",accuracy_score(y_train,clf.predict(X_train))*100,"%")
    print("La precision es",precision_score(y_train,clf.predict(X_train))*100, "%")
    print("El recall es",recall_score(y_train,clf.predict(X_train))*100, "%")
    print("El F1 Score es",f1_score(y_train,clf.predict(X_train))*100,"%")
    tn, fp, fn, tp=confusion_matrix(y_train,clf.predict(X_train)).ravel()
    print("tn:",tn," fp:",fp," fn:",fn," tp:",tp)
    print("Datos de test:")
    print("El accuracy es",accuracy_score(y_test,clf.predict(X_test))*100,"%")
    print("La precision es",precision_score(y_test,clf.predict(X_test))*100, "%")
    print("El recall es",recall_score(y_test,clf.predict(X_test))*100,"%")
    print("El F1 Score es",f1_score(y_test,clf.predict(X_test))*100,"%")
    tn_t, fp_t, fn_t, tp_t=confusion_matrix(y_test,clf.predict(X_test)).ravel()
    print("tn:",tn_t," fp:",fp_t," fn:",fn_t," tp:",tp_t)      

In [ ]:
#Vamos a definir la parrilla para realizar Randomized Grid Search

# Máximo número de niveles en el árbol. Damos una distribución con mayor probabilidad en valores pequeños
max_depth1 = [int(x) for x in np.linspace(2, 20, num = 10)]#"Sobreponderamos" árboles con poca profundidad
max_depth2 =[int(x) for x in np.linspace(30, 100, num = 4)]
max_depth=max_depth1 + max_depth2

# Mínimo número de observaciones en cada hoja. Damos una distribución con mayor probabilidad en los valores más elevados
min_samples_leaf_1 = [int(x) for x in np.linspace(5, 50, num = 4)]
min_samples_leaf_2 = [int(x) for x in np.linspace(60, 100, num = 10)]
min_samples_leaf=min_samples_leaf_1+min_samples_leaf_2

# Hemos asignado mayor número de casos a los parámetros que podrían facilitar la explicabilidad del Decision Tree  

In [ ]:
# Creamos la grid aleatoria
random_grid = {'max_depth': max_depth,
               'min_samples_leaf': min_samples_leaf
               }

In [ ]:
clf_tree = DecisionTreeClassifier(random_state=0)
#Creamos una versión para optimizar la precision
clf_tree_random_p= RandomizedSearchCV(random_state=0,estimator = clf_tree, param_distributions = random_grid, n_iter = 100, cv = 5,scoring="precision")
#Creamos una versión para optimizar el F1 Score
clf_tree_random_f1= RandomizedSearchCV(random_state=0,estimator = clf_tree, param_distributions = random_grid, n_iter = 100, cv = 5,scoring="f1")

In [ ]:
#Comenzamos por el DataFrame X (el X3 anterior)
#Generamos conjuntos de train y el de test. Para el test usamos el 20% de las observaciones
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
clf_tree_random_p.fit(X_train, y_train)

In [ ]:
print("tuned hpyerparameters :(best parameters) ",clf_tree_random_p.best_params_)

In [ ]:
#Trabajamos ahora con el mejor modelo encontrado en el Randomized Search, optimizando la precision. Entrenamos ahora el modelo
#con todos los datos de train y luego lo evaluaremos con un conjunto de test no utilizado en la estimación
clf_tree_p = clf_tree_random_p.best_estimator_
clf_tree_p.fit(X_train,y_train)

In [ ]:
eval_modelo_2 (clf_tree_p,X_train,y_train, X_test,y_test)

In [ ]:
X.columns[clf_tree_p.feature_importances_>0.10]
#Las variables que obtenemos de este árbol que parece que produce overfitting son las que habíamos visto anteriormente 

In [ ]:
#Vemos ahora los resultados con la optimización realizada sobre F1 Score

clf_tree_random_f1.fit(X_train, y_train)

print("tuned hpyerparameters :(best parameters) ",clf_tree_random_f1.best_params_)

clf_tree_f1 = clf_tree_random_f1.best_estimator_  
clf_tree_f1.fit(X_train,y_train)

eval_modelo_2 (clf_tree_f1,X_train,y_train, X_test,y_test)

In [ ]:
#Tomamos el árbol anterior como árbol de referencia ya que ofrece el mejor balance de Precision-Recall
clf_tree_best=clf_tree_f1

In [ ]:
X.columns[clf_tree_f1.feature_importances_>0.10]

In [ ]:
#Parece que los modelos anteriores overfittean. Vamos a probar un modelo más sencillo
clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=3,random_state=0)
clf_tree.fit(X_train,y_train)
eval_modelo_2(clf_tree,X_train,y_train, X_test,y_test)

In [ ]:
clf_tree_benchmark=clf_tree

In [ ]:
dibu_arb(clf_tree_benchmark)

In [ ]:
print(X_train.columns[19])
print(X_train.columns[25])
print(X_train.columns[22])
print(X_train.columns[24])
print(X_train.columns[36])

In [ ]:
dibu_arb(clf_tree_best)
# El árbol que hemos obtenido a partir del Randomized Grid Search es mucho más complejo, y por tanto, difícil de interpretar 

In [ ]:
X.columns[clf_tree_f1.feature_importances_>0.10]
#A pesar de la mayor complejidad de este modelo, las variables que muestra como más importantes son las mismas que en el de
#max_depth=3

In [ ]:
#Vamos a ver si la manipulación de variables que habíamos realizado anteriormente puede mejorar el modelo
#Comenzamos por el DataFrame X_log
#Generamos conjuntos de train y de test. Para el test usamos el 20% de las observaciones
X_log_train, X_log_test, y_log_train, y_log_test = train_test_split(X_log, y, test_size=0.2, random_state=0)

In [ ]:
clf_tree_random_p.fit(X_log_train, y_log_train)
print("tuned hpyerparameters :(best parameters) ",clf_tree_random_p.best_params_)

In [ ]:
clf_tree_p = clf_tree_random_p.best_estimator_  
clf_tree_p.fit(X_log_train,y_log_train)
eval_modelo_2(clf_tree_p,X_log_train, y_log_train,X_log_test, y_log_test)

In [ ]:
X.columns[clf_tree_p.feature_importances_>0.10]

In [ ]:
#Optimizamos ahora el F1 Score
clf_tree_random_f1.fit(X_log_train, y_log_train)
print("tuned hpyerparameters :(best parameters) ",clf_tree_random_f1.best_params_)

In [ ]:
clf_tree_f1 = clf_tree_random_f1.best_estimator_  
clf_tree_f1.fit(X_log_train, y_log_train)
eval_modelo_2 (clf_tree_f1,X_log_train, y_log_train,X_log_test, y_log_test)

In [ ]:
X_log.columns[clf_tree_f1.feature_importances_>0.10]

In [ ]:
#Parece que los modelos anteriores overfittean. Vamos a probar un modelo más sencillo
clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=3,random_state=0)
clf_tree.fit(X_log_train, y_log_train)
eval_modelo_2(clf_tree,X_log_train, y_log_train,X_log_test, y_log_test)

In [ ]:
X_log.columns[clf_tree_f1.feature_importances_>0.10]

In [ ]:
#A continuación seguimos con el DataFrame X_bin, en el que hemos binarizado las variables que contaban el número de veces que
#produce una operativa
#Generamos conjuntos de train y de test. Para el test usamos el 20% de las observaciones
X_bin_train, X_bin_test, y_bin_train, y_bin_test = train_test_split(X_bin, y, test_size=0.2, random_state=0)

In [ ]:
clf_tree_random_p.fit(X_bin_train, y_bin_train)
print("tuned hpyerparameters :(best parameters) ",clf_tree_random_p.best_params_)

In [ ]:
clf_tree_p = clf_tree_random_p.best_estimator_  
clf_tree_p.fit(X_bin_train,y_bin_train)
eval_modelo_2(clf_tree_p,X_bin_train, y_bin_train,X_bin_test, y_bin_test)

In [ ]:
X_bin.columns[clf_tree_p.feature_importances_>0.10]

In [ ]:
#Optimizamos ahora el F1 Score
clf_tree_random_f1.fit(X_bin_train, y_bin_train)
print("tuned hpyerparameters :(best parameters) ",clf_tree_random_f1.best_params_)

In [ ]:
clf_tree_f1 = clf_tree_random_f1.best_estimator_  
clf_tree_f1.fit(X_bin_train, y_bin_train)
eval_modelo_2 (clf_tree_f1,X_bin_train, y_bin_train,X_bin_test, y_bin_test)

In [ ]:
X_bin.columns[clf_tree_f1.feature_importances_>0.10]

In [ ]:
#Utilizamos la misma matriz de hiperparámetros
grid=random_grid

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
k=StratifiedKFold(n_splits=5, random_state=0,shuffle=False)
#Probamos con esta técnica por si el hecho que la Precision salga mejor optimizando F1 Score que optimizando Precision es por
#algún factor aleatorio al aplicar Cross Validation

In [ ]:
#Creamos los modelos para hacer la búsqueda de hiperparámetros 
clf_tree = DecisionTreeClassifier(random_state=0)
#Creamos una versión para optimizar la precision
clf_tree_gridsearch_p= GridSearchCV(estimator = clf_tree, param_grid = grid, cv=k, scoring="precision")
#Creamos una versión para optimizar el F1 Score
clf_tree_gridsearch_f1= GridSearchCV(estimator = clf_tree, param_grid = grid, cv=k, scoring="f1")

In [ ]:
#Vemos ahora los resultados con la optimización realizada sobre Precision

clf_tree_gridsearch_p.fit(X_train, y_train)

print("tuned hpyerparameters :(best parameters) ",clf_tree_gridsearch_p.best_params_)

clf_tree_gs_p = clf_tree_gridsearch_p.best_estimator_  
clf_tree_gs_p.fit(X_train,y_train)

eval_modelo_2 (clf_tree_gs_p,X_train,y_train, X_test,y_test)

In [ ]:
#Veamos las features de mayor relevancia en el modelo
X.columns[clf_tree_gs_p.feature_importances_>0.10]

In [ ]:
#Vemos ahora los resultados con la optimización realizada sobre F1 Score

clf_tree_gridsearch_f1.fit(X_train, y_train)

print("tuned hpyerparameters :(best parameters) ",clf_tree_gridsearch_f1.best_params_)

clf_tree_gs_f1 = clf_tree_gridsearch_f1.best_estimator_
clf_tree_gs_f1.fit(X_train,y_train)

eval_modelo_2 (clf_tree_gs_f1,X_train,y_train, X_test,y_test)

In [ ]:
#Veamos la relevancia de las features en el modelo
clf_tree_gs_f1.feature_importances_

In [ ]:
#Vemos que muchas features tienen relevancia cero  o prácticamente nula 
#Veamos las features de mayor relevancia en el modelo
X.columns[clf_tree_gs_f1.feature_importances_>0.10]

In [ ]:
dibu_arb(clf_tree_gs_f1)

# Analizamos a continuación la interpretabilidad de los resultados obtenidos 

In [ ]:
#pip install pdpbox

In [ ]:
from pdpbox import pdp

In [ ]:
#conda install -c conda-forge shap 

In [ ]:
import shap
shap.initjs()

In [ ]:
#Comenzamos el análisis de los SHAP values
explainer = shap.TreeExplainer(clf_tree_gs_f1)

In [ ]:
#Preparamos los SHAP values para el train
shap_values_train = explainer.shap_values(X_train)

In [ ]:
#Preparamos tambien los SHAP values para el test
shap_values_test = explainer.shap_values(X_test)

# Análisis de variable Num_Type_VYBER

In [ ]:
#Vemos el impacto de 'Num_Type_VYBER' en X_train con Partial Dependece Plot
pdp_feature = pdp.pdp_isolate(model=clf_tree_gs_f1, dataset=X_train, model_features=X.columns, feature='Num_Type_VYBER')
pdp.pdp_plot(pdp_feature, 'Num_Type_VYBER')
plt.show()

In [ ]:
#Vemos el impacto de 'Num_Type_VYBER' en X_test con Partial Dependece Plot
pdp_feature = pdp.pdp_isolate(model=clf_tree_gs_f1, dataset=X_test, model_features=X.columns, feature='Num_Type_VYBER')
pdp.pdp_plot(pdp_feature, 'Num_Type_VYBER')
plt.show()

In [ ]:
#Veamos la distribución de la variable y en cuantas observaciones tiene efecto esta feature
plt.hist(X['Num_Type_VYBER'])

In [ ]:
X[X['Num_Type_VYBER']>0].shape[0]

In [ ]:
#Para asegurar la consistencia del análisis de Partial Dependence vamos a analizar la correlación de esta variable con el resto
#de variables
X.corr()["Num_Type_VYBER"]

In [ ]:
#SHAP Values para  conjunto de train. En los desplegables de X e Y hay que seleccionar la variable Num_Type_VYBER (X) y
# Num_Type_VYBER effects (Y)
shap.force_plot(explainer.expected_value[1], shap_values_train[1], X_train)

In [ ]:
#SHAP Values para  conjunto de test. En los desplegables de X e Y hay que seleccionar la variable Num_Type_VYBER (X) y
# Num_Type_VYBER effects (Y)
shap.force_plot(explainer.expected_value[1], shap_values_test[1], X_test)

In [ ]:
plt.scatter(X.Num_Type_VYBER, y)
plt.xlabel("Num_Type_VYBER")
plt.ylabel("account_loan_bin")
plt.show()

In [ ]:
plt.scatter(X.Num_Type_VYBER, X.Ord_Leasing)
plt.xlabel("Num_Type_VYBER")
plt.ylabel("Ord_Leasing")
plt.show()

# Análisis de variable Num_Op_Remittances


In [ ]:
#Vemos el impacto de 'Num_Op_Remittances' en X_train con Partial Dependece Plot
pdp_feature = pdp.pdp_isolate(model=clf_tree_gs_f1, dataset=X_train, model_features=X.columns, feature='Num_Op_Remittances')
pdp.pdp_plot(pdp_feature, 'Num_Op_Remittances')
plt.show()

In [ ]:
#Vemos el impacto de 'Num_Op_Remittances' en X_test con Partial Dependece Plot
pdp_feature = pdp.pdp_isolate(model=clf_tree_gs_f1, dataset=X_test, model_features=X.columns, feature='Num_Op_Remittances')
pdp.pdp_plot(pdp_feature, 'Num_Op_Remittances')
plt.show()

In [ ]:
#Veamos la distribución de la variable y en cuantas observaciones tiene efecto esta feature
plt.hist(X['Num_Op_Remittances'])

In [ ]:
X[X['Num_Op_Remittances']>0].shape[0]

In [ ]:
#Para evaluar la consistencia del análisis de Partial Dependence vamos a analizar la correlación de esta variable con el resto
#de variables
X.corr()["Num_Op_Remittances"]

In [ ]:
#SHAP Values para conjunto de train. En los desplegables de X e Y hay que seleccionar la variable Num_Op_Remittances (X) y 
# Num_Op_Remittances effects (Y)

shap.force_plot(explainer.expected_value[1], shap_values_train[1], X_train)

In [ ]:
#SHAP Values para  conjunto de test. En los desplegables de X e Y hay que seleccionar la variable Num_Op_Remittances (X) y
# Num_Op_Remittances effects (Y)
shap.force_plot(explainer.expected_value[1], shap_values_test[1], X_test)

In [ ]:
plt.scatter(X.Num_Op_Remittances, y)
plt.xlabel("Num_Op_Remittances")
plt.ylabel("account_loan_bin")
plt.show()

# Análisis de variable Ord_Leasing

In [ ]:
#Vemos el impacto de 'Ord_Leasing' en X_train con Partial Dependece Plot
pdp_feature = pdp.pdp_isolate(model=clf_tree_gs_f1, dataset=X_train, model_features=X.columns, feature='Ord_Leasing')
pdp.pdp_plot(pdp_feature, 'Ord_Leasing')
plt.show()

In [ ]:
pdp_feature = pdp.pdp_isolate(model=clf_tree_gs_f1, dataset=X_test, model_features=X.columns, feature='Ord_Leasing')
pdp.pdp_plot(pdp_feature, 'Ord_Leasing')
plt.show()

Para el caso de Ord_Leasing, vemos que el tener este tipo de operativa disminuye la probabilidad de haber contratado un préstamo en un 10% aproximandamente.

In [ ]:
#Veamos la distribución de la variable y en cuantas observaciones tiene efecto esta feature
plt.hist(X['Ord_Leasing'])

In [ ]:
X[X['Ord_Leasing']>0].shape[0]

In [ ]:
#Para evaluar la consistencia del análisis de Partial Dependence vamos a analizar la correlación de esta variable con el resto
#de variables
X.corr()["Ord_Leasing"]

In [ ]:
plt.scatter(X.Num_Type_VYBER, X.Ord_Leasing)
plt.xlabel("Num_Type_VYBER")
plt.ylabel("Ord_Leasing")
plt.show()

In [ ]:
#SHAP Values para  conjunto de train. En los desplegables de X e Y hay que seleccionar la variable Ord_Leasing (X) y
#Ord_Leasing effects (Y)
shap.force_plot(explainer.expected_value[1], shap_values_train[1], X_train)

In [ ]:
#SHAP Values para  conjunto de test. En los desplegables de X e Y hay que seleccionar la variable Ord_Leasing (X) y
#Ord_Leasing effects (Y)
shap.force_plot(explainer.expected_value[1], shap_values_test[1], X_test)

In [ ]:
plt.scatter(X.Ord_Leasing, y)
plt.xlabel("Ord_Leasing")
plt.ylabel("account_loan_bin")
plt.show()

# "Calculadora" de SHAP values para las observaciones de test 

In [ ]:
#Para cada observación i podemos ver el impacto de las distintas features en su probabilidad de haber contratado un préstamo
i=52
shap.force_plot(explainer.expected_value[1], shap_values_test[1][i,:], feature_names=X_test.columns)

In [ ]:
#Para cada observación i podemos ver el valor de las variables que hemos seleccionado (las más significativas del modelo)
X_test.iloc[i][['Num_Type_VYBER','Num_Op_Remittances','Ord_Leasing']]

# A continuación  vamos a ver qué resultados obtendríamos con Random Forest (un modelo más potente también basado en árboles)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#Comenzamos a probar con un Random Forest sencillo: 
clf_RF= RandomForestClassifier(max_depth=30,min_samples_leaf=5,random_state=0,n_estimators=50)
clf_RF.fit(X_train,y_train)
eval_modelo_2 (clf_RF,X_train,y_train, X_test,y_test)

In [ ]:
X.columns[clf_RF.feature_importances_>0.10]

In [ ]:
#Replicamos el Random Forest obtenido en la optimización anterior
clf_RF_gs_f1= RandomForestClassifier(max_depth=14,min_samples_leaf=5,n_estimators=200,random_state=0)
clf_RF_gs_f1.fit(X_train, y_train)
eval_modelo_2 (clf_RF_gs_f1,X_train,y_train, X_test,y_test)

In [ ]:
#Vemos a continuación las variables más relevantes para el modelo:
X.columns[clf_RF_gs_f1.feature_importances_>0.10]
#Estas variables coinciden con las que habíamos analizado anteriormente

In [ ]:
#Replicamos el Random Forest obtenido en la optimización anterior
clf_RF_gs_f1= RandomForestClassifier(max_depth=30,min_samples_leaf=5,n_estimators=200,random_state=0)
clf_RF_gs_f1.fit(X_train, y_train)
eval_modelo_2 (clf_RF_gs_f1,X_train,y_train, X_test,y_test)

In [ ]:
#En el caso de optimizar el F1 Scores con número de estimadores=200, las features más relevantes siguen siendo 
#las que habíamos visto anteriormente:
X.columns[clf_RF_gs_f1.feature_importances_>0.10]

In [ ]:
#Replicamos el Random Forest obtenido en la optimización anterior
clf_RF_gs_f1= RandomForestClassifier(max_depth=30,min_samples_leaf=5,n_estimators=500,random_state=0)
clf_RF_gs_f1.fit(X_train, y_train)
eval_modelo_2 (clf_RF_gs_f1,X_train,y_train, X_test,y_test)

In [ ]:
#Las features más relevantes siguen siendo las que habíamos visto anteriormente:
X.columns[clf_RF_gs_f1.feature_importances_>0.10]

# A continuación  vamos a ver que resultados obtendríamos con un modelo más potente, tambien basado en árboles, como Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
#Cosideramos un primer modelo para probar
clf_GBC = GradientBoostingClassifier(max_depth=4,random_state=0)
clf_GBC.fit(X_train,y_train)
eval_modelo_2 (clf_GBC,X_train,y_train, X_test,y_test)

In [ ]:
#Veamos cuales son las variables más significativas para este modelo 
X.columns[clf_GBC.feature_importances_>0.1]

In [ ]:
#Muy fácilmente obtenemos modelos que aprenden el 100% de los datos de Train, aunque en este caso, claramente tiene overfitting
clf_GBC = GradientBoostingClassifier(max_depth=10,random_state=0)
clf_GBC.fit(X_train,y_train)
eval_modelo_2 (clf_GBC,X_train,y_train, X_test,y_test)

In [ ]:
#Muy fácilmente obtenemos modelos que aprenden el 100% de los datos de Train (sólo variando un parámetro)
clf_GBC = GradientBoostingClassifier(n_estimators=1000,random_state=0)
clf_GBC.fit(X_train,y_train)
eval_modelo_2 (clf_GBC,X_train,y_train, X_test,y_test)

In [ ]:
#A partir de los datos obtenidos por la optimización reproducimos el modelo
clf_GBC_gs_f1 = GradientBoostingClassifier(learning_rate= 0.2,max_depth=3,min_samples_leaf= 15,
                                                       n_estimators= 1000,random_state=0)
clf_GBC_gs_f1.fit(X_train,y_train)

eval_modelo_2 (clf_GBC_gs_f1,X_train,y_train, X_test,y_test)

In [ ]:
#Veamos cuales son las variables más significativas para este modelo 
X.columns[clf_GBC_gs_f1.feature_importances_>0.09]
#Bajamos un poco el umbral para ver si sale exactamente lo mismo que en el caso de Decision Tree y en este caso entraría
#una nueva variable

In [ ]:
#Siguiendo las indicaciones de la página https://github.com/slundberg/shap, vemos que no hay una implementación rápida 
#para los modelos de Gradient Boosting Classifier y debemos utilizar la implementación genérica
explainer = shap.KernelExplainer(clf_GBC_gs_f1.predict_proba,X_train, link="logit")
shap_values = explainer.shap_values(X_test,nsamples=2)
#Incluimos un nsamples bajo para ver que el código se puede ejecutar pero no lo ejecutamos con nsamples alto porque tarda mucho
#en ejecutarse

In [ ]:
#Con pocas samples, SHAPE values no funciona muy bien (no muestra valores), pero a continuación muestro una "calculadora" 
#para poder analizar el efecto de cada variable en una observación dada
i=10
shap.force_plot(explainer.expected_value[1], shap_values[1][52,:], X_test.iloc[i,:])

In [ ]:
#Vemos el impacto de 'Ord_Household_Payment_amount' en X_train con Partial Dependece Plot
pdp_feature = pdp.pdp_isolate(model=clf_GBC_gs_f1, dataset=X_test, model_features=X.columns, feature='Ord_Household_Payment_amount')
pdp.pdp_plot(pdp_feature, 'Ord_Household_Payment_amount')
plt.show()

In [ ]:
#Vemos el impacto de 'Ord_Household_Payment_amount' en X_test con Partial Dependece Plot
pdp_feature = pdp.pdp_isolate(model=clf_GBC_gs_f1, dataset=X_test, model_features=X.columns, feature='Ord_Household_Payment_amount')
pdp.pdp_plot(pdp_feature, 'Ord_Household_Payment_amount')
plt.show()

In [ ]:
#Veamos la distribución de la variable y en cuantas observaciones tiene efecto esta feature
plt.hist(X['Ord_Household_Payment_amount'])

In [ ]:
X[X['Ord_Household_Payment_amount']>0].shape[0]

In [ ]:
#Para asegurar la consistencia del análisis de Partial Dependence vamos a analizar la correlación de esta variable con el resto
#de variables
X.corr()["Ord_Household_Payment_amount"]

In [ ]:
plt.scatter(X.Ord_Household_Payment_amount, y)
plt.xlabel("Ord_Household_Payment_amount")
plt.ylabel("account_loan_bin")
plt.show()

In [ ]:
#Vemos el impacto de 'Num_Type_VYBER' en X_train con Partial Dependece Plot
pdp_feature = pdp.pdp_isolate(model=clf_GBC_gs_f1, dataset=X_test, model_features=X.columns, feature='Num_Type_VYBER')
pdp.pdp_plot(pdp_feature, 'Num_Type_VYBER')
plt.show()

In [ ]:
#Vemos el impacto de 'Num_Type_VYBER' en X_test con Partial Dependece Plot
pdp_feature = pdp.pdp_isolate(model=clf_GBC_gs_f1, dataset=X_test, model_features=X.columns, feature='Num_Type_VYBER')
pdp.pdp_plot(pdp_feature, 'Num_Type_VYBER')
plt.show()

In [ ]:
#Vemos el impacto de 'Num_Op_Remittances' en X_train con Partial Dependece Plot
pdp_feature = pdp.pdp_isolate(model=clf_GBC_gs_f1, dataset=X_test, model_features=X.columns, feature='Num_Op_Remittances')
pdp.pdp_plot(pdp_feature, 'Num_Op_Remittances')
plt.show()

In [ ]:
#Vemos el impacto de 'Num_Op_Remittances' en X_test con Partial Dependece Plot
pdp_feature = pdp.pdp_isolate(model=clf_GBC_gs_f1, dataset=X_test, model_features=X.columns, feature='Num_Op_Remittances')
pdp.pdp_plot(pdp_feature, 'Num_Op_Remittances')
plt.show()

In [ ]:
clf_GBC_gs_f1.feature_importances_

In [ ]:
clf_tree_gs_f1.feature_importances_